# Pràctica 2: Recomanador Simple

Nom dels alumnes del grup: **Martí Martínez Gargallo i Diego Hernández Antón**


## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** usuari, amb id's entre 1 i 6040 
- **MovieID** pel·licula, amb id's entre 1 i 3952
- **Rating** puntuació, en una escala de 1 a 5 estrelles.
- **Timestamp** representat en segons

Cada usuari té com a mínim 20 puntuacions.

### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, alguns usuaris poden no tenir informació.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes estan separats i seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies. 

## 2. Exploració de les dades

### 2.1. Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

In [1]:
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [2]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [3]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')

### 2.2. Inspecció de les taules

In [4]:
users[:10]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [5]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [6]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [7]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [8]:
ratings.sort_values('movie_id')[:5]

,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [9]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 **Exemple:** Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [11]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
28501,1,48,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
13819,1,938,4,978301752,F,1,10,48067,Gigi (1958),Musical
51327,1,1207,4,978300719,F,1,10,48067,To Kill a Mockingbird (1962),Drama
31152,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37916,1,2762,4,978302091,F,1,10,48067,"Sixth Sense, The (1999)",Thriller
18472,1,2687,3,978824268,F,1,10,48067,Tarzan (1999),Animation|Children's
45685,1,2692,4,978301570,F,1,10,48067,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
22832,1,720,3,978300760,F,1,10,48067,Wallace & Gromit: The Best of Aardman Animatio...,Animation
32771,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [12]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [13]:
# comptem quin tant per cent de ratings estan fets per una dona

print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes: 
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [14]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [15]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

Reviseu aquest concepte: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [16]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64


Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [17]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [18]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [19]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [20]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [21]:
print(np.nan + 9.0) 

nan


Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [22]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [23]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [24]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python i actueu en conseqüència:

In [25]:
%timeit data['title'] 
print(type(data['title']))
%timeit data.title 
print(type(data.title))
%timeit data[['title']] 
print(type(data[['title']]))

5.81 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
<class 'pandas.core.series.Series'>
14.2 µs ± 44.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
<class 'pandas.core.series.Series'>
34.4 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``. 

In [26]:
data_folder = 'ml-1m'

In [27]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

# la vostra solució aquí
users_mean_rating = pd.DataFrame(data.groupby('user_id')['rating'].mean())
users_mean_rating

,rating
user_id,
1,4.188679
2,3.713178
3,3.901961
4,4.190476
5,3.146465
...,...
6036,3.302928
6037,3.717822
6038,3.800000


+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ). 

In [28]:
# la vostra solució aquí
"""Fem mitjana de les valoracions agrupant per títol"""
movie_mean_rating = data.groupby('title')['rating'].mean()
best_movie_rating = movie_mean_rating.idxmax()
max_rating = movie_mean_rating[best_movie_rating]
print(f"Millor puntuació en mitjana: {max_rating}")
print(f"Pel·lícula més ben puntuada en mitjana: {best_movie_rating}")

Millor puntuació en mitjana: 5.0
Pel·lícula més ben puntuada en mitjana: Baby, The (1973)


+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada.

> **Resposta:** Clarament n'hi ha més:

In [29]:
# la vostra solució aquí
best_movies = movie_mean_rating[movie_mean_rating == max_rating].index
best_movies

Index(['Baby, The (1973)', 'Bittersweet Motel (2000)',
       'Follow the Bitch (1998)', 'Gate of Heavenly Peace, The (1995)',
       'Lured (1947)', 'One Little Indian (1973)',
       'Schlafes Bruder (Brother of Sleep) (1995)', 'Smashing Time (1967)',
       'Song of Freedom (1936)', 'Ulysses (Ulisse) (1954)'],
      dtype='object', name='title')

+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Així tindrem la pel·licula més ben puntuada per més usuaris. 

In [30]:
# la vostra solució aquí
"""
Cada fila de data correspon a una valoració, 
comptem quantes corresponen a un títol.
"""
best_movie_rating_maxviews = data.groupby('title').size()[best_movies].idxmax()
best_movie_rating_maxviews

'Gate of Heavenly Peace, The (1995)'

### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.


In [31]:
"""
Trobem el títol de la pel·lícula a partir de l'índex de 
la fila que correspon al registre major puntuat de l'usuari
"""
def top_movie(dataFrame,usr):
    # la vostra solució aquí
    return dataFrame.iloc[dataFrame[dataFrame['user_id'] == usr]['rating'].idxmax()]['title']
    
print(top_movie(data,1))

One Flew Over the Cuckoo's Nest (1975)


### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors siguin el rating donat.

In [32]:
"""
Haurem de vigilar perquè movies pot tenir duplicats.
"""
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí
    return df.pivot(values='rating', index='user_id', columns='movie_id')

In [33]:
df_counts = build_counts_table(data)
df_counts

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


+ Fés una funció que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [34]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """
    
    # la vostra solució aquí
    return df.loc[user_id, movie_id]

get_count(df_counts, 1, 1)

5.0

### 3.4. EXERCICI D

In [35]:
data.nunique()

user_id         6040
movie_id        3706
rating             5
timestamp     458455
gender             2
age                7
occupation        21
zip             3439
title           3706
genres           301
dtype: int64

In [36]:
unique_movies = pd.unique(data['movie_id'])
unique_movies.max()

3952

Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039. 

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

> **Resposta**: Hi ha 3706 identificadors diferents però el seu valor màxim és 3952 (s'assoleix un valor major del nombre d'índexos: no tenim identificadors consecutius). Amb això veiem que no els tenim des de 0 fins a 3705 com ens agradaria: caldrà modificar-los d'alguna manera (igual que haurem de restar un a l'índex dels usuaris per aconseguir els índexos de 0 a 6039).


+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [37]:
# la vostra solució aquí
"""Obtenim els codis categòrics d'usuaris i pel·lícules i els usem per reindexar."""
data['user_id'] = pd.Categorical(data['user_id']).codes
data['movie_id'] = pd.Categorical(data['movie_id']).codes

+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [38]:
# la vostra solució aquí
df_counts = build_counts_table(data)
df_counts

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la distància Euclidiana entre dos vectors usant funcions de pandas. 

+ Escriu la funció `SimEuclid (DataFrame, U1, U2)` que calculin la semblança entre dos usuaris segons aquesta estructura:

    + Calcular un vector per cada usuari, $U1$ i $U2$, amb les puntuacions dels ítems comuns que han puntuat el dos usuaris.
    + Si no hi ha puntuacions en comú, retornar 0. Sinó, retornar 
    
    $$\frac{1}{(1+distEuclid(U1, U2))}$$

+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [39]:
num_movies = data.nunique()['movie_id']

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """
    
    # la vostra solució aquí
    return np.linalg.norm(x-y)

def SimEuclid(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana
    
    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """
    
    # la vostra solució aquí
    """
    Trobem mitjançant una màscara les pel·lícules puntuades en comú.
    Notem que per fer això cal usar la funció np.isnan() de NumPy.
    """
    U1 = DataFrame.loc[User1].values
    U2 = DataFrame.loc[User2].values
    mask = ~np.isnan(U1) & ~np.isnan(U2)
    U1 = U1[mask]
    U2 = U2[mask]
    
    """Ponderem la similaritat en base al nombre de pel·lícules puntuades en comú."""
    return len(U1) * (1 / (1 + distEuclid(U1, U2))) / num_movies

In [40]:
# Execute functions
print(SimEuclid(df_counts, 2,314))

0.0005396654074473826


In [41]:
# Execute functions
print(SimEuclid(df_counts, 0,1))

0.0006296096420219463


In [42]:
# Execute functions
print(SimEuclid(df_counts, 1, 5))

0.0005316828473355942


In [43]:
%timeit SimEuclid(df_counts, 1, 5)

64.5 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de semblança (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. 

Exemple: ``getRecommendationsUser(data, 2, 50, 10, SimEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (userID) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score*, que seran els que usarem per fer la recomanació. Normalitzeu els *scores* de sortida.

In [44]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari d'usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """
    
    # la vostra solució aquí
    """Creem un DataFrame amb els usuaris (excepte userID), calculem la similaritat respecte userID i prenem les m majors."""
    scores = pd.DataFrame(index=DataFrame.index, columns=['similarity'])
    scores.drop(userID, inplace=True)
    scores = scores.apply(lambda x:SimEuclid(DataFrame,userID,x.name), axis=1).sort_values(ascending=False).head(m)
    
    """Normalitzem les similituds perquè sumin 1 i retornem el resultat en un diccionari si hi ha algun usuari comparable."""
    suma = scores.sum()
    return {} if suma == 0 else (scores/suma).to_dict()

In [45]:
t = datetime.datetime.now()
sim_dict = find_similar_users(df_counts, 2, 10, SimEuclid)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.422316


In [46]:
sim_dict

{4447: 0.10868178502623507,
 1646: 0.10748625569533282,
 5830: 0.10106784766151948,
 2270: 0.09904834121693383,
 1903: 0.09883902906949076,
 3271: 0.09823974081452751,
 4276: 0.09823495519138713,
 1879: 0.09634838620212503,
 3625: 0.09614044342242399,
 650: 0.0959132157000244}

+ Creieu que el temps de procés és assumible? Quan trigaria si ho fem per tots els usuaris?

In [47]:
"""Temps (en minuts) de processar tots els usuaris tenint en compte que per cadascun triguem al voltant de 0.45 segons."""
len(df_counts)*0.45/60

45.3

> **RESPOSTA**: Com hem optimitzat el càlcul de la similitud no obtenim una quantitat extremadament gran de temps. De fet, sí seria assumible esperar tres quarts d'hora per processar les dades si no ho haguéssim de fer gaires cops (si per exemple recalculéssim les similaritats a la nit pel dia següent). Tot i això, podem millorar aquest valor com veurem a continuació!

+ Per solucionar el problema anterior, construeix una matriu de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. Calcula quant triga la teva implementació. 

Compareu aquestes dues opcions des del punt de vista de temps de càlcul:

* Feu una funció que construeixi ``similarity_matrix1`` a partir d'un doble ``for`` sobre els usuaris i que calculi la distància entre usuaris com la distància entre els subvectors formats pels elements en comú dels dos usuaris.
* Feu una funció que construeixi ``similarity_matrix2`` *aproximadament*:
 * Substituïnt els ``NaN`` corresponents a ítems no avaluats per ``0``.
 * Treballant específicament amb operacions matricials. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/



In [48]:
def compute_distance(fixed_arr, var_arr):
    """
    Donats dos vectors, calcula la distància entre els subvectors formats 
    pels elements en comú. 
    """
    
    # la vostra solució aquí
    """Fem el mateix que a SimEuclid per dos vectors concrets."""
    mask = ~np.isnan(fixed_arr) & ~np.isnan(var_arr)
    fixed_arr = fixed_arr[mask]
    var_arr = var_arr[mask]
    
    """Ponderem la similaritat en base al nombre de pel·lícules puntuades en comú."""
    return len(fixed_arr) * (1 / (1 + distEuclid(fixed_arr, var_arr))) / num_movies

In [49]:
def similarity_matrix1(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param df_counts: df amb els valors que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    # la vostra solució aquí
    """
    Fem un doble for per calcular les similaritats aprofitant la simetria i ometent la diagonal.
    V: Matriu amb les valoracions. M: Nombre d'usuaris.
    """
    V = df_counts.values
    M = V.shape[0]
    S = np.zeros((M,M))
    for i in range(M-1):
        for j in range(i+1,M):
            sim = compute_distance(V[i],V[j])
            S[i,j] = sim
            S[j,i] = sim
    return S

In [50]:
t = datetime.datetime.now()
sim_for = similarity_matrix1(compute_distance, df_counts)
t1 = datetime.datetime.now()-t
print("Temps amb doble for:",str(t1))

Temps amb doble for: 0:04:09.423033


Veiem que en només 4 minuts tenim totes les similaritats entre usuaris. Ara, si volem trobar usuaris similars per fer una recomanació simplement consultarem aquesta matriu! 

D'altra banda, podem millorar aquest temps amb operacions matricials (**vam comentar a classe que el càlcul matricial, si substituïm les valoracions inexistents per 0, dona valors de similaritat aproximats. Hem trobat, però, una manera d'operar matricialment obtenint, a més, els valors correctes. Es triga uns 3 cops més que amb la similaritat aproximada però en una mica més de 4 segons ho tenim**).

Notem que podem expressar la distància euclidiana entre els vectors de valoracions dels usuaris $i,j$ de la següent manera (on $I$ és el nombre d'ítems, pel·lícules en el nostre cas, i $v_s$ és el vector de les valoracions de l'usuari $s$):

$$\text{dist}_{ij} = \sqrt{\sum_{k=0}^{I-1} (v_{ik} - v_{jk})^2} =  \sqrt{\sum_{k=0}^{I-1} v_{ik}^2 - 2v_{ik}v_{jk} + v_{jk}^2} = \sqrt{v_i·v_i-2v_i·v_j+v_j·v_j}$$
             
El problema amb els NaN és que, si l'usuari $i$ ha puntuat la pel·lícula 1 però l'usuari $j$ no, per $k = 1$ tindrem que el terme $v_{ik}^2$ contribueix al sumatori mentres que $- 2v_{ik}v_{jk} + v_{jk}^2$ és 0. És a dir, estarem sumant un terme quadràtic que no correspon (aquest és el motiu pel qual es va afirmar que el resultat matricial és aproximat i no exacte). Per solucionar-ho, determinarem la matriu booleana $B = (b_{il})$ corresponent a les valoracions existents dels usuaris (1 si per l'usuari $i$ s'ha valorat la pel·lícula $l$, 0 en cas que el valor sigui NaN). Transposant-la i multiplicant matricialment per la matriu formada pels quadrats de la matriu de valoracions $V$ ($V*V$, on el producte és element a element), obtenim la següent matriu:

$X = (x_{ij})_{i,j=0,...,M-1}$, on $x_{ij} = \sum_{k=0}^{I-1} v_{ik}^2b_{jk}$, on $M$ és el nombre d'usuaris

Per tant, aquesta matriu tindrà, a la posició $(i,j)$, la suma dels quadrats de les valoracions de l'usuari $i$ d'aquelles pel·lícules puntuades tant per ell com per l'usuari $j$. Amb això ja tenim el problema resolt: a la matriu $X$ tenim, a la fila $i$, columna $j$, el producte escalar $v_i·v_i$ (valoracions usuari $i$) amb només les valoracions de les pel·lícules puntuades per l'usuari $j$. Transposem $X$ per poder sumar, a la posició $(i,j)$, el producte escalar $v_j·v_j$ (valoracions usuari $j$) amb únicament les valoracions de les pel·lícules puntuades per l'usuari $i$. Pels productes creuats ($XY$) fem el producte matricial $VV^T$.

Finalment, per obtenir la similaritat ponderada segons la longitud del vector de pel·lícules vistes en comú per dos usuaris, multipliquem matricialment $BB^T$:

$ BB^T = (w_{ij})_{i,j=0,...,M-1}$, on $w_{ij} = \sum_{k=0}^{I-1} b_{ik}b_{jk}$

És a dir, a la posició $(i,j)$ obtenim quantes pel·lícules tenen puntuades en comú els usuaris $i,j$, doncs $b_{ik}b_{jk}$ només és 1 (*True*) quan ambdós factors son 1.

Anem a codificar aquest *approach*, mirarem quant triguem i compararem element a element amb la matriu de similaritat obtinguda amb ``similarity_matrix1``, per veure si estem calculant les similaritats correctament (i molt més ràpid!).

In [51]:
"""Obtenim df_counts amb els NaN a 0."""
df_counts = df_counts.fillna(0)

In [52]:
def similarity_matrix2(df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    # la vostra solució aquí
    """Determinem les matrius V, B, X, XY esmentades amunt."""
    V = df_counts.values
    B = (V!=0).astype(float)
    X = np.matmul(V*V, B.T)
    XY = np.matmul(V, V.T)
    
    """Operem, notem que cal restar la identitat, doncs la similaritat entre d'un usuari amb ell mateix és 1."""
    return np.matmul(B, B.T) * ((1/(1 + np.sqrt(X - 2*XY + X.T))) - np.identity(V.shape[0])) / num_movies

In [53]:
t = datetime.datetime.now()
sim_mat = similarity_matrix2(df_counts)
t2 = datetime.datetime.now()-t
print("Operant matricialment:",str(t2))

Operant matricialment: 0:00:04.292906


In [54]:
"""Comprovem que la similaritat amb el doble for ens dona el mateix que amb el mètode matricial implementat."""
(sim_for == sim_mat).all()

True

Així, a partir d'ara treballarem amb la matriu de similaritat obtinguda usant la funció ``similarity_matrix2`` (tenim el mateix resultat que amb el doble *for* en 4 minuts menys):

In [55]:
sim = sim_mat

+ Ara torna a re-fer la funció ``find_similar_users`` i mira quant triga. 

> Recorda que les scores han d'estar normalitzades!

In [56]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param sim_mx: matriu de similaritat
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :return : dictionary
    """
    
    # la vostra solució aquí
    """Ordenem les similaritats i prenem els m usuaris que la maximitzen."""
    index = np.nonzero(DataFrame.index == userID)[0][0]
    I = np.argsort(sim_mx[index])[-m:]
    scores = sim_mx[index,I]
    
    """Suma 0 => Cap usuari similar. En cas contrari normalitzem i convertim a diccionari."""
    suma = scores.sum()
    return {} if suma == 0 else dict(zip(DataFrame.iloc[I].index,scores/suma))

In [57]:
t = datetime.datetime.now()
sim_dict = find_similar_users(df_counts, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.001791


In [58]:
sim_dict

{650: 0.0959132157000244,
 3625: 0.09614044342242399,
 1879: 0.09634838620212503,
 4276: 0.09823495519138713,
 3271: 0.09823974081452751,
 1903: 0.09883902906949076,
 2270: 0.09904834121693383,
 5830: 0.10106784766151948,
 1646: 0.10748625569533282,
 4447: 0.10868178502623507}

Notem que aquest diccionari és el mateix que l'obtingut amb la primera versió de ``find_similar_users`` (tot i que aquí tenim el diccionari en ordre creixent segons les similaritats normalitzades i abans era decreixent). Notem que, de 0.422316 segons hem passat a 0.001791.

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels ratings dels seus $m$ usuaris més propers. 
 * Fes primer una funció ``weighted_average`` que retorni un diccionari del tipus ``{peli_id: score_predit}`` amb la puntuació predita de cada ítem a partir de les puntuacions dels $m$ usuaris més propers i de la seva semblança a l'usuari considerat.
 * Fes després una funció ``getRecommendationsUser`` que usant la funció anterior retorni un ``df`` amb els $n$ ítems amb més score i els seus scores.

In [59]:
def weighted_average(DataFrame, user, sim_mx, m):
    """
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_matrix
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return: diccionari {peli_id: score_predit}
    """
    
    # la vostra solució aquí
    """
    U: m usuaris més similars. S: Similaritat entre usuaris d'U i user. 
    R: Rating de pel·lícules ponderat amb les similaritats.
    """
    U = find_similar_users(DataFrame, sim_mx, user, m)
    S = np.array(list(U.values())).reshape(-1,1)
    R = S*DataFrame.loc[np.array(list(U.keys()))].values
    
    """
    Prenem la suma de les valoracions ponderades per cada film. Per tal de poder comparar 
    aquestes valoracions cal normalitzar per la suma de les similaritats dels usuaris implicats.
    """
    scores = np.zeros(R.shape[1])
    sim_sum = np.sum(S * (R!=0), axis=0)
    mask = sim_sum != 0
    scores[mask] = np.sum(R,axis=0)[mask] / sim_sum[mask]
    return dict(zip(DataFrame.columns,scores))

In [60]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : dataframe de pel·lícules amb els scores predits
    """
    
    # la vostra solució aquí
    """
    weighted_average retorna un diccionari amb els scores predits per cada pel·lícula (s).
    Només cal passar aquesta informació a un DataFrame i prendre els n films amb major score.
    """
    s = weighted_average(DataFrame, user, sim_mx, m)
    return pd.DataFrame(data=s.values(),index=s.keys(),columns=['score']).sort_values(by='score',ascending=False).head(n)

In [61]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(df_counts, 3, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.008527


In [62]:
user_prediction

,score
721,5.0
3287,5.0
2008,5.0
909,5.0
1984,5.0
1950,5.0
3600,5.0
643,5.0
1757,5.0
1753,5.0


### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 
+ La llista amb els ratings originals d'un usuari donat
+ La llista de les prediccions generades per aquest usuari

#### EXERCICI G.1

+ Treu el 10% dels usuaris i reserva aquests en una variable anomenada ``test_set`` i la resta en una variable anomenada ``train_set``.

In [63]:
# la vostra solució aquí
"""Fixem el valor de la llavor per no obtenir inconsistències amb el text redactat en cas de reexecutar el notebook."""
np.random.seed(654)

"""Obtenim una mostra del 10% dels usuaris (test_set) i els eliminem del conjunt de dades total (train_set)."""
test_rate = 0.1
users = data['user_id'].unique()
test_set = data[data['user_id'].isin(np.random.choice(users, size=int(len(users)*test_rate), replace=False))]
train_set = data.drop(test_set.index)

assert len(test_set) + len(train_set) == len(data)

+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**: Que no rebrem cap predicció rellevant perquè no tenim cap dada dels usuaris del conjunt de test per entrenar.

#### EXERCICI G.2

+ Selecciona aproximadament el 80% de les interaccions de cada usuari de test i junta-les al ``train_set``. Podem ara podem evaluar el sistema?


> Com la pràctica és molt llarga us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afagi el 80% de les intraccions i les unifiqui al dataframe de train.

> **Resposta**: Ara sí podrem. Veurem quina aproximació tenim del 20% d'interaccions que no veu el conjunt d'entrenament dels usuaris de test.

In [64]:
test_set.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,14,1104,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
5,17,1104,4,978156168,F,18,3,95825,One Flew Over the Cuckoo's Nest (1975),Drama
6,18,1104,5,982730936,M,1,10,48073,One Flew Over the Cuckoo's Nest (1975),Drama
23,87,1104,5,977694161,F,45,1,02476,One Flew Over the Cuckoo's Nest (1975),Drama
24,88,1104,5,977683596,F,56,9,85749,One Flew Over the Cuckoo's Nest (1975),Drama


In [65]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

user_id
10      27.4
13       5.0
14      40.2
17      61.0
18      51.0
        ... 
5997    31.6
6024    54.2
6029    23.0
6030    10.2
6037     4.0
Name: movie_id, Length: 604, dtype: float64

Seleccionem la posició 1 i aquest user_id serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [66]:
groupby_count.reset_index().iloc[1]

user_id     13.0
movie_id     5.0
Name: 1, dtype: float64

In [67]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [68]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

TOTAL SAMPLES OF THE USER: 25
TOTAL SAMPLES OF THE USER IN TEST SET: 5


In [69]:
len(test_set_user.index)

25

In [70]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

TOTAL SAMPLES OF THE USER IN TRAIN SET: 20


In [71]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [72]:
def add_testdata(traindf, test_set):
    """
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return : 
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """
    
    # la vostra solució aquí
    """Fem allò tractat abans però per tots els usuaris del conjunt de test. Ajuntem el 80% de test_set a train."""
    test_rate = 0.2
    train = pd.DataFrame(traindf)
    test = pd.DataFrame()
    groupby_count = test_set.groupby('user_id')['movie_id'].count()*test_rate
    for u,n in zip(groupby_count.index, groupby_count.astype(int)):
        test_user = test_set[test_set['user_id'] == u]
        frame_test = test_user.sample(n)
        train = pd.concat([train, test_user[~test_user.index.isin(frame_test.index)]])
        test = pd.concat([test, frame_test])
        
    return train, test

In [73]:
train, test = add_testdata(train_set, test_set)

In [74]:
train

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,0,1104,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,1104,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,11,1104,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
4,16,1104,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama
7,23,1104,5,978136709,F,25,7,10023,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
799455,6037,3314,4,956707604,F,56,1,14706,Auntie Mame (1958),Comedy|Drama
810947,6037,2872,5,956707640,F,56,1,14706,Harvey (1950),Comedy
815636,6037,1094,5,956717204,F,56,1,14706,"English Patient, The (1996)",Drama|Romance|War
836349,6037,1316,4,956714815,F,56,1,14706,Walkabout (1971),Drama


In [75]:
test

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
549762,10,1928,5,978903658,F,25,1,04093,"Jerk, The (1979)",Comedy
14388,10,2710,5,978903129,F,25,1,04093,Ferris Bueller's Day Off (1986),Comedy
111821,10,501,3,978220867,F,25,1,04093,"Remains of the Day, The (1993)",Drama
471534,10,1993,4,978903278,F,25,1,04093,Beetlejuice (1988),Comedy|Fantasy
210644,10,1059,4,978902965,F,25,1,04093,Monty Python and the Holy Grail (1974),Comedy
...,...,...,...,...,...,...,...,...,...,...
239639,6030,46,3,956718439,F,18,0,45123,Seven (Se7en) (1995),Crime|Thriller
656926,6037,225,4,956707640,F,56,1,14706,Eat Drink Man Woman (1994),Comedy|Drama
212239,6037,1059,4,956707708,F,56,1,14706,Monty Python and the Holy Grail (1974),Comedy
189932,6037,1007,5,956707547,F,56,1,14706,"Fish Called Wanda, A (1988)",Comedy


In [76]:
train.shape

(981020, 10)

In [77]:
test.shape

(19189, 10)

In [78]:
data.shape

(1000209, 10)

In [79]:
assert train.shape[0] + test.shape[0] == data.shape[0]

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE. 

In [80]:
def evaluateRecommendations(train, test, m, n, sim):
    """
    Retorna l'error generat pel model
    
    :param train: dataframe que conté totes les dades d'entrenament
    :param test: dataframe que conté totes les dades de test
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a recomanar als usuaris
    :param sim: matriu de similaritat
    :return : Escalar (float) corresponent al MAE
    """
   
    # la vostra solució
    """
    Mirem les pel·lícules recomanades per cada usuari. Si està valorada (al conjunt de test), determinem quina és la 
    diferència entre la predicció i el rating real. Fem una mitjana aritmètica del valor absolut dels errors (MAE).
    """
    suma = 0
    count = 0
    for u in test.index:
        """
        Per cada usuari determinem quines pel·lícules s'han predit i valorat simultàniament. D'aquestes, determinem
        la diferència en valor absolut (diffs). Afegim a les variables de la mitjana els valors no NaN d'aquest vector.
        """
        predictions = getRecommendationsUser(train, u, sim, n, m)
        ratings = test.loc[u]
        I = np.intersect1d(predictions.index, ratings.index)
        diffs = np.abs(predictions.loc[I].values.reshape(len(I)) - ratings.loc[I].values)
        diffs = diffs[~np.isnan(diffs)]
        suma += np.sum(diffs)
        count += len(diffs)
        
    """Si no hi cap coincidència entre prediccions i valoracions retornem np.nan per no confondre amb MAE=0."""
    return np.nan if count == 0 else suma/count

In [81]:
"""
Primer determinem df_counts i similarity_matrix a partir del df train. Volem també df_counts de test 
per saber quins son els ratings reals a avaluar. Notem que aquests df_counts NO contenen necessàriament 
tots els usuaris ni totes les pel·lícules! Caldrà tenir-ho en compte (per exemple, usar .loc i no .iloc).
"""
df_counts_train = build_counts_table(train).fillna(0)
df_counts_test = build_counts_table(test)

t = datetime.datetime.now()
sim_train = similarity_matrix2(df_counts_train)
mae = evaluateRecommendations(df_counts_train, df_counts_test, 50, 10, sim_train)
t = datetime.datetime.now()-t
print(f"Temps en construir la matriu de similaritat i calcular el MAE: {t}")

Temps en construir la matriu de similaritat i calcular el MAE: 0:00:07.209506


In [82]:
mae

0.29486984879418654

Així, en mitjana, l'error en el *rating* de les pel·lícules recomanades és inferior a 0.3.

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.

Per veure si podria sortir més a compte farem dos recomanadors: un per cada gènere (preferim aquesta terminologia a sexe perquè a les dades originals la columna s'anomena ``gender``) a partir del conjunt d'entrenament i test generats a l'apartat anterior. Per cadascun d'ells, determinarem el MAE (*Mean Absolute Error*) obtingut (tal i com hem fet a l'anterior exercici). Si és un valor similar o superior al del recomanador únic (variable ``mae``), podrem estar segurs que no guanyarem res amb un recomanador específic, doncs això comportarà un esforç addicional (càlcul de 2 recomanadors, haver de consultar si l'usuari és home i dona cada cop que vulguem recomanar...) sense ser profitós.

In [83]:
"""df_counts d'entrenament i test per cada gènere."""
train_m = build_counts_table(train[train['gender'] == 'M']).fillna(0)
test_m = build_counts_table(test[test['gender'] == 'M'])
train_f = build_counts_table(train[train['gender'] == 'F']).fillna(0)
test_f = build_counts_table(test[test['gender'] == 'F'])

"""Matrius de similaritat."""
sim_train_m = similarity_matrix2(train_m)
sim_train_f = similarity_matrix2(train_f)

"""MAE dels recomanadors específics."""
mae_m = evaluateRecommendations(train_m, test_m, 50, 10, sim_train_m)
mae_f = evaluateRecommendations(train_f, test_f, 50, 10, sim_train_f)

"""Resultats."""
print(f"MAE recomanador genèric: {mae}\nMAE recomanador masculí: {mae_m}\nMAE recomanador femení: {mae_f}")

MAE recomanador genèric: 0.29486984879418654
MAE recomanador masculí: 0.5000000000000009
MAE recomanador femení: nan


Comparant-ho amb l'experiment de l'exercici G obtenim un resultat pitjor en el gènere masculí. De fet, la ràtio 0.5/0.29487 és propera a 1.7, indicant-nos que l'error s'incrementa en, aproximadament, un 70%. D'altra banda, les pel·lícules recomanades en el gènere femení no coincideixen, en cap cas, amb les que teníem per avaluar al conjunt de test. Per tant, no només el recomanador específic ho fa pitjor (segons aquesta tria de conjunt d'entrenament i test, almenys) al gènere masculí, sinó que amb el femení ni tan sols podem determinar com de bo és. Això té sentit si pensem que al dividir el recomanador en 2 tenim moltes menys dades per treballar en cada cas. 

En conclusió, tal i com hem comentat a l'inici de l'exercici, en la nostra opinió **no surt a compte** fer un recomanador específic per cada gènere. A més, no sabem com està la legislació en aquest àmbit però no ens sembla gaire ètic usar el gènere per recomanar, doncs això implica considerar els gustos de cadascun per separat (i, per tant, diferenciant-los).